In [2]:
# libraries

import time
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from colorama import Fore, Back, Style

np.set_printoptions(precision = 2, suppress = True)

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

In [3]:
np.random.seed(1234)

### The TRANSFIL model

In [5]:
class Data:
    def __init__(self, pop_size):
        self.initialize()
        
        self.pop_size = pop_size
        self.WF = np.zeros(self.pop_size)
        self.WM = np.zeros(self.pop_size)
        self.Mf = np.zeros(self.pop_size)
        
        # Simulate a random age between 0 and 100 years
        self.age = np.random.uniform(0, 100*12, self.pop_size)
        self.age = np.floor(self.age)
        self.time = 0;
        
        # Draw the bite risk parameter
        self.k = np.random.uniform(self.min_k, self.max_k)
        
        # Draw probabilities of treatment
        self.beta_shape1 = (1 - self.systematic_compliance) / self.systematic_compliance
        self.beta_shape2 = (1 - self.treatment_coverage)*(1 - self.systematic_compliance)/self.systematic_compliance;
        self.treatment_prob = np.random.beta(self.beta_shape1, self.beta_shape2, self.pop_size);

        self.excluded = np.random.binomial(1, self.sys_ex, self.pop_size);
        # ----------- IntegerVector ttreat_(pop_size, 15*12);
        self.ttreat = np.ones(self.pop_size) * (15*20)
        self.bednet = np.random.binomial(1, self.bednet_coverage, self.pop_size);
        self.bite_risk = np.random.gamma(self.k, 1/self.k, self.pop_size);
        self.vth = np.random.uniform(self.min_vth, self.max_vth);
        self.r = np.random.uniform(self.min_r, self.max_r);
        self.importation_rate = np.random.uniform(self.min_importation_rate, self.max_importation_rate);
        self.alpha = np.random.uniform(self.min_alpha, self.max_alpha);
        self.kappa = np.random.uniform(self.min_kappa, self.max_kappa);

        self.burn_in_done = False
        
  
    def initialize(self):
        # Host
        self.tau = 0.00167; # death rate
        
        self.bednet_coverage = 0;
        self.bednet_reduction = 0.97;

        self.min_k = 0.01;
        self.max_k = 1.6;

        self.importation_rate = 0;
        self.min_importation_rate = 0;
        self.max_importation_rate = 0.0025;
        self.max_importation_rate = 0.005;

        # Vector
        self.lambda_ = 10.0; # bite rate mosq/month
        # proportion of mosquitoes which pick up infection when biting an infected host
        self.g = 0.37; 
        self.larvae = 5.0; # average l3 density
        self.sigma = 5.0; # mosquito death rate
        # double kappa = 4.395; # L3 update and development for anophales
        
        self.min_kappa = 4.395;
        self.max_kappa = 4.395;
        self.min_r = 0.055;
        self.max_r = 0.055;
        self.species = 1; # 0 for Culex, 1 for anopheles
        self.s_N = 0.03; # probability of successful feeding
        self.d_N = 0.41; # probability of death

        # Worm
        self.psi_1 = 0.414; # Proportion L3 leaving mosquito per bite
        self.psi_2 = 0.32; # Proportion of L3 leaving mosquito that enter host
        self.s_2 = 0.00275; # Proportion of L3 entering host that develop into adults
        self.mu = 0.0104; # worm death rate
        self.gamma = 0.1; # mf death rate
        
        self.min_alpha = 0.58;
        self.max_alpha = 0.58;

        self.min_vth = 1.0;
        self.max_vth = 180.0;

        self.treatment_coverage = 0.01; # Treatment coverage
        self.systematic_compliance = 0.3; # Correlation between rounds
        self.fecundity = 9; # month of worm infertility due to MDA
        self.sys_ex = 0.05; # Proportion of the population that are excluded
        self.prop_mf_killed = 0.99;
        self.prop_worm_killed = 0.65;

        # prevalence vectors
        self.worm_prev = [];
        self.mf_prev = [];
        
        self.start = 0
        
    def run_animation(self):
        animation_values = ["10%", "20%", "30%", "40%", "50%", "60%", "70%", "80%", "90%", "100%"]
        animation = ["[■□□□□□□□□□]","[■■□□□□□□□□]", "[■■■□□□□□□□]", "[■■■■□□□□□□]", "[■■■■■□□□□□]", "[■■■■■■□□□□]", "[■■■■■■■□□□]", "[■■■■■■■■□□]", "[■■■■■■■■■□]", "[■■■■■■■■■■]"]
        runtime = np.floor(np.linspace(1,self.n_months,10))
        
        if self.time == 1: print("Loading...")
        
        if self.time in runtime:
            sys.stdout.write("\r" + Fore.RED + animation[self.start] + " " + Fore.BLACK + animation_values[self.start])
            sys.stdout.flush()
            self.start += 1


    def timestep(self, n_months):
        self.n_months = n_months
        for i in range(self.n_months):
            self.run_MDA()
            self.update_bednet()
            self.worm_dynamics()
            self.mf_dynamics()
            self.larvae_dynamics()
            self.calculate_prevalence()
            self.aging()
            self.importation()
            if self.burn_in_done:
                self.run_animation()
        if self.burn_in_done: 
            print("\nFinished simulation.")
        
    def worm_dynamics(self):
        verbose = False
        if verbose: print("Updating worm dynamics\n")
        for i in range(self.pop_size):
            age_prop = self.age[i]/(9*12)
            bite_rate = min(age_prop,1)
            BNR = self.s_N if self.bednet[i] else 1
            birth_rate = 0.5 * self.lambda_ * self.bite_risk[i] * self.vth * self.psi_1 * self.psi_2 * self.s_2 * bite_rate * self.larvae * BNR
            
            if verbose:
                print(f"i = {i}, birth_rate = {birth_rate}")
                print(f" lambda = {self.lambda_}, bite_risk = {self.bite_risk[i]}, "
                      f"vth = {self.vth}, psi_1 = {self.psi_1}, psi_2 = {self.psi_2}, "
                      f"s_2 = {self.s_2}, bite_rate = {bite_rate}, larvae = {self.larvae}, "
                      f"BNR = {BNR}")
                print(f"Mean bite risk = {sum(self.bite_risk) / len(self.bite_risk)}")
            
            male_death_rate = self.WM[i] * self.mu
            female_death_rate = self.WF[i] * self.mu

            male_births = np.random.poisson(birth_rate) if birth_rate > 0 else 0
            female_births = np.random.poisson(birth_rate) if birth_rate > 0 else 0
            male_deaths = np.random.poisson(male_death_rate) if male_death_rate > 0 else 0
            female_deaths = np.random.poisson(female_death_rate) if female_death_rate > 0 else 0

            self.WM[i] += male_births - male_deaths
            self.WF[i] += female_births - female_deaths
            
            if self.WM[i] < 0:
                self.WM[i] = 0
            
            if self.WF[i] < 0:
                self.WF[i] = 0

    def mf_dynamics(self):
        verbose = False
        if verbose:
            print("Updating MF dynamics with alpha = ", self.alpha, " and gamma = ", self.gamma)
            
        for i in range(self.pop_size):
            self.Mf[i] *= 1 - self.gamma
            if self.WM[i] == 0: continue
            if self.ttreat[i] <= self.fecundity: continue
            self.Mf[i] += self.alpha * self.WF[i]
            if self.Mf[i] < 0: self.Mf[i] = 0

    def larvae_dynamics(self):
        verbose = False
        L3 = []
        if self.species == 0:
            # Culex
            L3 = self.kappa * (1 - np.exp(-self.Mf * self.r / self.kappa))
        elif self.species == 1:
            # Anophales
            L3 = self.kappa * (1 - np.exp(-self.Mf * self.r / self.kappa))**2

        mean_larvae = sum(L3 * self.bite_risk) / sum(self.bite_risk)
        lambda_bednet = self.lambda_ * (self.bednet_coverage * (self.s_N - 1) + 1)
        sigma_bednet = self.sigma + (self.sigma * self.d_N * self.bednet_coverage)
        self.larvae = lambda_bednet * self.g * mean_larvae / (sigma_bednet + lambda_bednet * self.psi_1)

        if verbose:
            print(f"""Updating larvae dynamics with lambda_bednet = {lambda_bednet}, g = {self.g}, mean_larvae = {mean_larvae}, 
                  sigma_bednet = {sigma_bednet}, 
                  psi_1 = {self.psi_1} - lambda = {self.lambda_}, 
                  bednet_coverage = {self.bednet_coverage}, 
                  s_N = {self.s_N}""")

    
    def calculate_prevalence(self):
        verbose = False
        worm_pos = 0

        for i in range(self.pop_size):
            if self.age[i] < 12 * 5:
                continue
            if self.WM[i] > 0 or self.WF[i] > 0:
                worm_pos += 1

        mf_ind = np.random.uniform(0, 1, self.pop_size) < 1 - np.exp(-1 * self.Mf)

        current_wormprev = 0 if worm_pos == 0 else worm_pos / self.pop_size * 100
        self.worm_prev.append(current_wormprev)
        self.mf_prev.append(np.mean(mf_ind) * 100)

        # adjust importation rate if MDA has happened
        if not np.any(self.MDA_time == self.time):
            return
        prev_len = len(self.worm_prev)
        prev_before = self.worm_prev[prev_len - 2]
        if prev_before == 0:
            return
        prevalence_ratio = self.worm_prev[prev_len - 1] / prev_before
        if verbose:
            print(f"Prevalence ratio = {prevalence_ratio}, prev before = {self.worm_prev[prev_len - 2]}, "
                  f"prev after = {self.worm_prev[prev_len - 1]}, importation rate before = {self.importation_rate}, "
                  f"importation rate after = {self.importation_rate * prevalence_ratio}")
        self.importation_rate *= prevalence_ratio

    
    def aging(self):
        self.age += 1
        self.ttreat += 1
        self.time += 1

        host_reset = (np.random.uniform(0, 1, self.pop_size) < 1 - np.exp(-self.tau)) | (self.age > 1200)

        num_reset = sum(host_reset)

        if num_reset == 0:
            return

        self.age[host_reset] = 0
        self.WF[host_reset] = 0
        self.WM[host_reset] = 0
        self.Mf[host_reset] = 0

        self.bite_risk[host_reset] = np.random.gamma(self.k, 1/self.k, num_reset)
        self.ttreat[host_reset] = 15 * 12
        self.bednet[host_reset] = np.random.binomial(1, self.bednet_coverage, num_reset)
        
        beta_shape1 = self.treatment_coverage * (1 - self.systematic_compliance) / self.systematic_compliance
        beta_shape2 = (1 - self.treatment_coverage) * (1 - self.systematic_compliance) / self.systematic_compliance
 
        if beta_shape1 == 0 or beta_shape2 == 0:
            self.treatment_prob[host_reset] = np.zeros(num_reset)
        else:
            self.treatment_prob[host_reset] = np.random.beta(beta_shape1, beta_shape2, num_reset)

        exclude = host_reset & (np.random.uniform(0, 1, self.pop_size) < self.sys_ex)
        self.excluded[exclude] = 0
    
    
    def importation(self):
        importation_ind = (np.random.uniform(0.0, 1.0, self.pop_size) < 1 - np.exp(-1 * self.importation_rate))
        if not np.any(importation_ind):
            return

        mean_larvae = 10
        new_worms = int(np.floor(0.5 * self.lambda_ * self.vth * self.psi_1 * self.psi_2 * self.s_2 * mean_larvae))
        # print("new worms = ", new_worms)
        self.WF[importation_ind] = new_worms
        self.WM[importation_ind] = new_worms
        self.Mf[importation_ind] = 0

    def set_bednet(self, bednet_data):
        verbose = False
        self.bednet_time = bednet_data["time"]
        self.bednet_coverage_vec = bednet_data["coverage"]
        
        if verbose:
            print("Bednet data set at times {}, with coverage {}".format(self.bednet_time, self.bednet_coverage_vec))
        
    
    def set_MDA(self, MDA_data):
        verbose = False
        self.MDA_time = MDA_data["time"]
        self.MDA_coverage = MDA_data["coverage"]
        
        if verbose:
            print("Setting MDA at times ", self.MDA_time, " with coverage ", self.MDA_coverage)
        
    def set_parameter(self, parameter, value):
        if parameter == "k":
            self.k = value
            self.bite_risk = np.random.gamma(self.k, 1/self.k, self.pop_size)
            return

        if parameter == "vth":
            self.vth = value
            return

        if parameter == "r":
            self.r = value
            return

        if parameter == "alpha":
            self.alpha = value
            return

        if parameter == "importation_rate":
            self.importation_rate = value
            return
        
        if parameter not in ["k", "vth", "alpha", "importation_rate"]:
            print("Unknown parameter")
            exit()

    
    def run_MDA(self):
        if not any(self.MDA_time == self.time): return
        verbose = False
        if verbose:
            print(f"Running MDA at time t = {self.time}")

        for i in range(len(self.MDA_time)):
            if self.time != self.MDA_time[i]: continue
            if self.treatment_coverage == self.MDA_coverage[i]: continue

            # Redraw treatment probabilities
            self.treatment_coverage = self.MDA_coverage[i]
            beta_shape1 = self.treatment_coverage * (1 - self.systematic_compliance) / self.systematic_compliance
            beta_shape2 = (1 - self.treatment_coverage) * (1 - self.systematic_compliance) / self.systematic_compliance

            if beta_shape1 == 0 or beta_shape2 == 0:
                self.treatment_prob = np.zeros(self.pop_size)
            else:
                self.treatment_prob = np.random.beta(beta_shape1, beta_shape2, self.pop_size)

        U = np.random.uniform(0, 1, self.pop_size)
        treat = (U < self.treatment_prob)

        for i in range(self.pop_size):
            if self.excluded[i] == 1: continue
            if treat[i] == 0: continue
            if self.age[i] < 5 * 12: continue

            self.WM[i] = int(self.WM[i] * (1 - self.prop_worm_killed))
            self.WF[i] = int(self.WF[i] * (1 - self.prop_worm_killed))
            self.Mf[i] *= 1 - self.prop_mf_killed

            self.ttreat[i] = 0

        self.larvae_dynamics()

    def update_bednet(self):
        verbose = False
        if not np.any(self.bednet_time == self.time): return
    
        for i in range(len(self.bednet_time)):
            current_bednet_time = self.bednet_time[i]
            if current_bednet_time != self.time: continue
            
            self.bednet_coverage = self.bednet_coverage_vec[i]
            self.bednet = np.random.binomial(1, self.bednet_coverage, self.pop_size)
            
            if verbose:
                print("At time", time, ", updating bednets with new coverage =", bednet_coverage, "and bednet =", bednet)
        return
                
   
    def burn_in(self):
        print("Burning in ...")
        for i in range(1200):
            self.timestep(1)
            self.time = 0
        length = len(self.worm_prev)
        self.worm_prev = [self.worm_prev[length - 1]]
        self.mf_prev = [self.mf_prev[length - 1]]
        self.burn_in_done = True
    
    def save(self):
        out = {}
        out["time"] = self.time
        out["pop_size"] = self.pop_size
        out["tau"] = self.tau
        out["bite_risk"] = self.bite_risk
        out["bednet"] = self.bednet
        out["bednet_coverage"] = self.bednet_coverage
        out["bednet_reduction"] = self.bednet_reduction
        out["k"] = self.k
        out["importation_rate"] = self.importation_rate
        out["lambda"] = self.lambda_
        out["g"] = self.g
        out["larvae"] = self.larvae
        out["sigma"] = self.sigma
        out["kappa"] = self.kappa
        out["r"] = self.r
        out["psi_1"] = self.psi_1
        out["psi_2"] = self.psi_2
        out["s_2"] = self.s_2
        out["mu"] = self.mu
        out["gamma"] = self.gamma
        out["alpha"] = self.alpha
        out["vth"] = self.vth
        out["WF"] = self.WF
        out["WM"] = self.WM
        out["Mf"] = self.Mf
        out["age"] = self.age
        out["treatment_prob"] = self.treatment_prob
        out["ttreat"] = self.ttreat
        out["excluded"] = self.excluded
        out["treatment_coverage"] = self.treatment_coverage
        out["systematic_compliance"] = self.systematic_compliance
        out["fecundity"] = self.fecundity
        out["sys_ex"] = self.sys_ex
        out["prop_mf_killed"] = self.prop_mf_killed
        out["prop_worm_killed"] = self.prop_worm_killed
        out["MDA_time"] = self.MDA_time
        out["MDA_coverage"] = self.MDA_coverage
        out["worm_prev"] = self.worm_prev
        out["mf_prev"] = self.mf_prev

        return out.copy()
    
    def load(self, input_data_raw):
        input_data = input_data_raw.copy()  # Cloning the input data
        self.time = input_data["time"]
        self.pop_size = input_data["pop_size"]
        self.tau = input_data["tau"]
        self.bite_risk = input_data["bite_risk"]
        self.bednet = input_data["bednet"]
        self.bednet_coverage = input_data["bednet_coverage"]
        self.bednet_reduction = input_data["bednet_reduction"]
        self.k = input_data["k"]
        self.importation_rate = input_data["importation_rate"]
        self.lambda_ = input_data["lambda"]  # 'lambda' is a reserved keyword in Python
        self.g = input_data["g"]
        self.larvae = input_data["larvae"]
        self.sigma = input_data["sigma"]
        self.kappa = input_data["kappa"]
        self.r = input_data["r"]
        self.psi_1 = input_data["psi_1"]
        self.psi_2 = input_data["psi_2"]
        self.s_2 = input_data["s_2"]
        self.mu = input_data["mu"]
        self.gamma = input_data["gamma"]
        self.alpha = input_data["alpha"]
        self.vth = input_data["vth"]
        self.WF = input_data["WF"]
        self.WM = input_data["WM"]
        self.Mf = input_data["Mf"]
        self.age = input_data["age"]
        self.treatment_prob = input_data["treatment_prob"]
        self.ttreat = input_data["ttreat"]
        self.excluded = input_data["excluded"]
        self.treatment_coverage = input_data["treatment_coverage"]
        self.systematic_compliance = input_data["systematic_compliance"]
        self.fecundity = input_data["fecundity"]
        self.sys_ex = input_data["sys_ex"]
        self.prop_mf_killed = input_data["prop_mf_killed"]
        self.prop_worm_killed = input_data["prop_worm_killed"]
        self.MDA_time = input_data["MDA_time"]
        self.MDA_coverage = input_data["MDA_coverage"]
        self.worm_prev = input_data["worm_prev"]
        self.mf_prev = input_data["mf_prev"]

### Running the model

In [ ]:
mda_coverages = [0.671,0.708,0.721,0.759,0.734,0.816,0.723,0.798,0.521,0.514,0,0.836,0.811,0.807,0.785,0.9281,0.6002,0,0.8403,0,0]

bednet_coverage = np.ones(21) * 0.67

mda_time = np.arange(112,353,12)

bednet_time = np.arange(112,353,12)

mda_settings = {"time": mda_time, "coverage": mda_coverages}

bednet_settings = {"time": bednet_time, "coverage": bednet_coverage}

x = Data(10000)
x.set_bednet(bednet_settings)
x.set_MDA(mda_settings)
x.set_parameter("k", 0.072)
x.set_parameter("vth", 130)
x.set_parameter("importation_rate", 0.00025)
x.burn_in()
x.timestep(460)

Burning in ...
Loading...
[■■■■■■■■□□] 80%

### Plots

In [ ]:
'''
Plotting antigen data and validations
'''

fig, ax = plt.subplots(1, 1, figsize = (10,5))
fig.tight_layout()
parameters = x.save()
txt = "Parameters: k = {}, systematic compliance = {}, vector to host ratio = {}".format(parameters["k"], parameters["systematic_compliance"], parameters["vth"])
fig.text(0.5, -0.05, txt, ha = "center", weight = "bold")

ax.plot(x.worm_prev, color = 'blue', label = "antigen prevalence")
ax.set(xlabel = "time starts since intervention (yrs)", ylabel = "antigen prevalence (%)", xlim = (100,460))
#ax.set_xticks([i for i in range(0,len(x.worm_prev[100:]),60)], ['2000','2005','2010','2015','2020','2025','2030'])

#ax.axvline(x = 85, color = 'g', label = '2007', linestyle = '--',  dashes=(5, 10))
#ax.axvline(x = 253, color = 'C1', label = 'ET & TT period', linestyle = '--',  dashes=(5, 10))
ax.axhline(y = 2, color = 'red', linestyle = 'dotted', label = '2% elimination threshold as a PHP')

ax.grid(linewidth = 0.2)
ax.legend()

plt.show()